In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import resample
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, precision_recall_curve, average_precision_score
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel

2025-07-06 15:59:40.113087: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751817580.326544      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751817580.390337      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Load and clean data
df = pd.read_csv("/kaggle/input/dataset3/D3.csv")
if 'Unnamed: 0' in df.columns:
    df.drop('Unnamed: 0', axis=1, inplace=True)
df.dropna(subset=['post', 'status'], inplace=True)
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)

In [3]:
# Resample to balance classes (preserve counts printout)
max_count = df['status'].value_counts().max()
df_bal = pd.DataFrame()
for status, grp in df.groupby('status'):
    if len(grp) < max_count:
        grp_res = resample(grp, replace=True, n_samples=max_count, random_state=42)
    else:
        grp_res = grp
    df_bal = pd.concat([df_bal, grp_res])

df_bal.reset_index(drop=True, inplace=True)
print("After resampling:\n", df_bal['status'].value_counts())

df = df_bal

After resampling:
 status
EDAnonymous      6947
adhd             6947
alcoholism       6947
autism           6947
ptsd             6947
schizophrenia    6947
Name: count, dtype: int64


In [4]:
# Build dynamic label map and one-hot labels
target_names = sorted(df['status'].unique())
num_classes = len(target_names)
label_map = {label: idx for idx, label in enumerate(target_names)}
df['label'] = df['status'].map(label_map)

In [5]:
# Tokenize with BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
MAX_LEN = 128
enc = tokenizer(
    df['post'].astype(str).tolist(), padding='max_length', truncation=True,
    max_length=MAX_LEN, return_tensors='np'
)
input_ids = enc['input_ids']
attention_mask = enc['attention_mask']

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [6]:
# Prepare labels and tf.data dataset
labels = tf.keras.utils.to_categorical(df['label'], num_classes=num_classes)
dataset = tf.data.Dataset.from_tensor_slices(
    ({'input_ids': input_ids, 'attention_mask': attention_mask}, labels)
)


I0000 00:00:1751817832.758704      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1751817832.759405      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [7]:
# Train/val/test split
BATCH = 32
dataset = dataset.shuffle(buffer_size=len(df), seed=42)
n = len(df)
n_train = int(0.8 * n)
n_val = int(0.1 * n)
train_ds = dataset.take(n_train).batch(BATCH)
val_ds = dataset.skip(n_train).take(n_val).batch(BATCH)
test_ds = dataset.skip(n_train + n_val).batch(BATCH)

In [8]:
# Build model
def build_model():
    bert = TFBertModel.from_pretrained('bert-base-uncased')
    in_ids = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_ids')
    in_mask = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='attention_mask')
    x = bert(in_ids, attention_mask=in_mask).pooler_output
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    out = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    return tf.keras.Model(inputs=[in_ids, in_mask], outputs=out)

model = build_model()
model.summary()

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 128)]                0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, 128)]                0         []                            
 )                                                                                                
                                                                                                  
 tf_bert_model (TFBertModel  TFBaseModelOutputWithPooli   1094822   ['input_ids[0][0]',           
 )                           ngAndCrossAttentions(last_   40         'attention_mask[0][0]']      
                             hidden_state=(None, 128, 7                                       

In [9]:
# Compile model
optim = tf.keras.optimizers.Adam(learning_rate=5e-5, epsilon=1e-8)
metrics = ['accuracy']
for cls, idx in label_map.items():
    metrics += [
        tf.keras.metrics.Precision(name=f"prec_{cls}", class_id=idx),
        tf.keras.metrics.Recall(name=f"rec_{cls}", class_id=idx)
    ]

model.compile(
    optimizer=optim,
    loss='categorical_crossentropy',
    metrics=metrics
)

In [10]:
#  Train
epochs = 10
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)
]
hist = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    callbacks=callbacks
)

Epoch 1/10


I0000 00:00:1751817928.803856     103 service.cc:148] XLA service 0x798dab6314d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1751817928.804522     103 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1751817928.804558     103 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1751817928.909271     103 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1751817929.056663     103 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1043/1043 [==============================] - ETA: 0s - loss: 0.4229 - accuracy: 0.8584 - prec_EDAnonymous: 0.9352 - rec_EDAnonymous: 0.8409 - prec_adhd: 0.9073 - rec_adhd: 0.8023 - prec_alcoholism: 0.9652 - rec_alcoholism: 0.8875 - prec_autism: 0.9211 - rec_autism: 0.7930 - prec_ptsd: 0.9200 - rec_ptsd: 0.8138 - prec_schizophrenia: 0.9124 - rec_schizophrenia: 0.7797

/usr/local/lib/python3.11/dist-packages/tf_keras/src/engine/training.py:3098: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native TF-Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1043/1043 [==============================] - 952s 865ms/step - loss: 0.4229 - accuracy: 0.8584 - prec_EDAnonymous: 0.9352 - rec_EDAnonymous: 0.8409 - prec_adhd: 0.9073 - rec_adhd: 0.8023 - prec_alcoholism: 0.9652 - rec_alcoholism: 0.8875 - prec_autism: 0.9211 - rec_autism: 0.7930 - prec_ptsd: 0.9200 - rec_ptsd: 0.8138 - prec_schizophrenia: 0.9124 - rec_schizophrenia: 0.7797 - val_loss: 0.1705 - val_accuracy: 0.9499 - val_prec_EDAnonymous: 0.9967 - val_rec_EDAnonymous: 0.8992 - val_prec_adhd: 0.9554 - val_rec_adhd: 0.9580 - val_prec_alcoholism: 0.9508 - val_rec_alcoholism: 0.9858 - val_prec_autism: 0.9628 - val_rec_autism: 0.9147 - val_prec_ptsd: 0.9801 - val_rec_ptsd: 0.9330 - val_prec_schizophrenia: 0.9431 - val_rec_schizophrenia: 0.9497
Epoch 2/10
1043/1043 [==============================] - 905s 867ms/step - loss: 0.1484 - accuracy: 0.9562 - prec_EDAnonymous: 0.9758 - rec_EDAnonymous: 0.9551 - prec_adhd: 0.9603 - rec_adhd: 0.9494 - prec_alcoholism: 0.9842 - rec_alcoholism: 0.9755 - 

In [11]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model

In [13]:
from sklearn.model_selection import train_test_split

# Re-split df the same way for test set (same stratified logic)
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])
val_df, test_df   = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['label'])



In [16]:
# Tokenize the test_df only
enc_test = tokenizer(
    test_df['post'].astype(str).tolist(),
    padding='max_length',
    truncation=True,
    max_length=MAX_LEN,
    return_tensors='np'
)


In [17]:
# Run prediction using the in-memory trained model
test_inputs = {'input_ids': enc_test['input_ids'], 'attention_mask': enc_test['attention_mask']}
probs = model.predict(test_inputs, batch_size=BATCH)
y_pred_idx = probs.argmax(axis=1)

131/131 [==============================] - 41s 278ms/step


In [18]:
# Convert back to label names
inv_label_map = {v: k for k, v in label_map.items()}
bert_preds = [inv_label_map[i] for i in y_pred_idx]

In [19]:
# Create aligned DataFrame
df_bert = pd.DataFrame({
    'text': test_df['post'].tolist(),
    'true_label': test_df['status'].tolist(),
    'bert_pred': bert_preds
})

In [20]:
# Save to CSV
df_bert.to_csv('predictions_bert.csv', index=False)
print("✅ predictions_bert.csv saved with", len(df_bert), "rows")
print(df_bert.head())

✅ predictions_bert.csv saved with 4169 rows
                                                text     true_label  \
0  Can you see SZ in someone's eyes? An NT in one...  schizophrenia   
1  Veteran Face to the Name: Geoff's PTSD Started...           ptsd   
2  Is it possible to have more than one innervoic...  schizophrenia   
3  Meltdowns/sensory issues getting worse? Hi I'm...         autism   
4  Book recommendations about autism so I can und...         autism   

       bert_pred  
0  schizophrenia  
1           ptsd  
2  schizophrenia  
3         autism  
4         autism  
